In [1]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/ZgXoXqK2KLjRLxD/download', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO'],parse_dates=['FECHA_CORTE', 'FECHA_VACUNACION'])
df=pd.read_csv('vacunas_covid.csv', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_VACUNACION', 'DOSIS', 'DEPARTAMENTO'], parse_dates=['FECHA_CORTE', 'FECHA_VACUNACION'])
FECHA_CORTE=df['FECHA_CORTE'].unique()
FECHA_CORTE=pd.to_datetime(FECHA_CORTE[0]).strftime('%Y-%m-%d')

In [2]:
#DIARIO DOSIS 1 Y DOSIS 2
df_ambas_dosis=df[['FECHA_VACUNACION','DOSIS','SEXO']].groupby(['FECHA_VACUNACION','DOSIS']).count()
df_ambas_dosis=df_ambas_dosis.reset_index()
df_ambas_dosis=df_ambas_dosis.pivot(index='FECHA_VACUNACION', columns='DOSIS', values='SEXO')
df_ambas_dosis=df_ambas_dosis.rename_axis(None, axis=1)
df_ambas_dosis.columns=['DOSIS1','DOSIS2']
df_ambas_dosis=df_ambas_dosis.fillna(0).astype('int')
df_ambas_dosis

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-09,3842,0
2021-02-10,16595,0
2021-02-11,33848,0
2021-02-12,32341,0
2021-02-13,18899,0
...,...,...
2021-05-05,17678,3609
2021-05-06,31650,1539
2021-05-07,24364,9778


In [3]:
#ACUMULADO DOSIS 1 Y DOSIS 2
df_ambas_dosis_cum=df_ambas_dosis.cumsum()
df_ambas_dosis_cum

,DOSIS1,DOSIS2
FECHA_VACUNACION,,
2021-02-09,3842,0
2021-02-10,20437,0
2021-02-11,54285,0
2021-02-12,86626,0
2021-02-13,105525,0
...,...,...
2021-05-05,1279294,664983
2021-05-06,1310944,666522
2021-05-07,1335308,676300


In [4]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]


In [8]:
# ACUMULADO POR DEPARTAMENTO (DOSIS 2 - VACUNACION COMPLETA)
df_ambas_dosis_departamento=df[['DEPARTAMENTO', 'SEXO']].groupby(['DEPARTAMENTO']).count()
df_ambas_dosis_departamento=df_ambas_dosis_departamento.reset_index()
df_ambas_dosis_departamento.rename(columns = {'SEXO':'DOSIS2'}, inplace = True)
df_ambas_dosis_departamento_dosis2=df_ambas_dosis_departamento[['DEPARTAMENTO','DOSIS2']]
df_ambas_dosis_departamento_dosis2=df_ambas_dosis_departamento_dosis2.reset_index(drop=True)
df_ambas_dosis_departamento_dosis2['POBLACION']=col_poblacion
df_ambas_dosis_departamento_dosis2['INDICE']=round(df_ambas_dosis_departamento_dosis2['DOSIS2']/(df_ambas_dosis_departamento_dosis2['POBLACION']/100000)).astype('int')
df_ambas_dosis_departamento_dosis2=df_ambas_dosis_departamento_dosis2.set_index('DEPARTAMENTO')
df_ambas_dosis_departamento_dosis2

,DOSIS2,POBLACION,INDICE
DEPARTAMENTO,,,
AMAZONAS,23244,426806,5446
ANCASH,49617,1180638,4203
APURIMAC,40018,430736,9291
AREQUIPA,81950,1497438,5473
AYACUCHO,47682,668213,7136
CAJAMARCA,39452,1453711,2714
CALLAO,166797,1129854,14763
CUSCO,38752,1357075,2856
HUANCAVELICA,33505,365317,9171


In [13]:
# ACUMULADO POR GRUPO ETARIO (DOSIS 2 - VACUNACION COMPLETA)
bins = [18,20,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 19 años','20 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [900000,5700000,5300000,4400000,3300000,2300000,1300000,700000] 
df_edades = df
df_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_edades = df_edades[df_edades.DOSIS == 2].groupby(['GRUPO_ETARIO'])["DOSIS"].count().reset_index()
df_edades.rename(columns = {'DOSIS':'DOSIS2'}, inplace = True)
df_edades['POBLACION']=poblacion_por_grupo_etario
df_edades['PORCENTAJE']=round(df_edades['DOSIS2']/df_edades['POBLACION']*100,2)
df_edades=df_edades.set_index('GRUPO_ETARIO')
df_edades

,DOSIS2,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 19 años,10063,900000,1.12
20 a 29 años,126416,5700000,2.22
30 a 39 años,160768,5300000,3.03
40 a 49 años,119122,4400000,2.71
50 a 59 años,88114,3300000,2.67
60 a 69 años,55009,2300000,2.39
70 a 79 años,24225,1300000,1.86
80 años a más,59186,700000,8.46


In [14]:
df_ambas_dosis.to_csv('../resultados/dosis1y2.csv')
df_ambas_dosis_cum.to_csv('../resultados/acumulados1y2.csv')
df_ambas_dosis_departamento_dosis2.to_csv('../resultados/departamentos.csv')
df_edades.to_csv('../resultados/dosis2_por_edades.csv')